## 对于不同的softmax温度,对概率分布进行重新加权

In [1]:
import numpy as np

def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution)/temperature
    distribution = np.exp(distribution)
    return distribution/np.sum(distribution)

# 实现字符级的LSTM文本生成

## 下载并解析初始文本文件

In [2]:
import keras
import numpy as np
path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt'
)
text = open(path).read().lower()
print('Corpus length: ', len(text))

/home/gengfeng/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
/home/gengfeng/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/gengfeng/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/gengfeng/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


606208/600901 [==============================] - 11s 18us/step
Corpus length:  600893


## 将字符序列向量化

In [4]:
# 提取60个字符组成的序列
maxlen = 60
# 每3个字符采样一个新序列
step = 3
# 保存所提取的序列
sentences = list()
# 保存目标(即下一个序列第一个字符)
next_chars = list()

for i in range(0, len(text)-maxlen, step):
    sentences.append(text[i: i+maxlen])
    next_chars.append(text[i+maxlen])
print('Number of sequences:', len(sentences))

# 语料中唯一字符组成的列表
chars = sorted(list(set(text)))
print('Unique characters: ', len(chars))
char_indices = {char:chars.index(char) for char in chars}

print(char_indices)

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices.get(char)] = 1
    y[i, char_indices.get(next_chars[i])] = 1

Number of sequences: 200278
Unique characters:  57
{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '=': 22, '?': 23, '[': 24, ']': 25, '_': 26, 'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 'k': 37, 'l': 38, 'm': 39, 'n': 40, 'o': 41, 'p': 42, 'q': 43, 'r': 44, 's': 45, 't': 46, 'u': 47, 'v': 48, 'w': 49, 'x': 50, 'y': 51, 'z': 52, 'ä': 53, 'æ': 54, 'é': 55, 'ë': 56}
Vectorization...


## 构建网络

In [8]:
from keras import layers
from keras.models import Sequential

model = Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               95232     
_________________________________________________________________
dense_1 (Dense)              (None, 57)                7353      
Total params: 102,585
Trainable params: 102,585
Non-trainable params: 0
_________________________________________________________________


## 模型编译配置

In [9]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy'
)

## 给定模型预测,采样下一个字符的函数

In [10]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## 文本生成循环

In [15]:
import random
import os
import sys

for epoch in range(1, 60):
    
    print('epoch', epoch)
    # 将模型在数据上拟合一次
    model.fit(x, y, batch_size=128, epochs=1)
    # 随机选择一个文本种子
    start_index = random.randint(0, len(text)-maxlen-1)
    generated_text = text[start_index: start_index+maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    # 尝试一系列不用的采样温度
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('-------------- temperature:', temperature)
        sys.stdout.write(generated_text)
        # 从种子文本开始,生成400个字符
        for i in range(400):
            # 对目前生成的字符进行one-hot编码
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.0
            # 对下一个字符进行采样
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
        sys.stdout.write('\n')
            

epoch 1
Epoch 1/1
200278/200278 [==============================] - 66s 332us/step - loss: 1.4265
--- Generating with seed: "of reconciliation
necessarily found even among philosophers "
-------------- temperature: 0.2
of reconciliation
necessarily found even among philosophers of the sense of the property and the attentually the such a stands of the superiority of the present the spirit and solitude, the provided to the spirit of the spirit of the spirit of the sense of the factism, the spirit of the spirit of the provided to the spirit of the provided to the factist to the strong the present the provided to the more the more and the soul and the present the faction of 
-------------- temperature: 0.5
e more the more and the soul and the present the faction of the souther seried, which is even to morality. and the form of the supremetain that the most dences, and propers and wholly in the certain there and a necession of a manifested the form of sublime the frontimy the factism the per

/home/gengfeng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


ontention of the same the sense the most satisfaction of the words of the most super--and the supposing and standard of the superioring the sense of the fact in the sense of the statering and the staterms of the
-------------- temperature: 0.5
e fact in the sense of the statering and the staterms of the way of the word of the first and the many profoundt, in
which we seems to be all the wasture of the latter of the morality, it please.


13

=soulting to leade in a moral existed and contempt and opensauends of supposing the staterms of the batter of the propended to the other the sense of the own opinion of the many the are be innocentible of the great them may be considerate the really as the s
-------------- temperature: 1.0
ble of the great them may be considerate the really as the serfiear as
the wich
fligin to
themselves.


1e
=diplee, of all which is
wasveking tathajacts, which over
most disposing the ielding, the
matters, disposing before the
indidionionally and confession (for


200278/200278 [==============================] - 68s 338us/step - loss: 1.3867
--- Generating with seed: "day.



chapter iv. apophthegms and interludes


63. he who "
-------------- temperature: 0.2
day.



chapter iv. apophthegms and interludes


63. he who have a south of the strength of the man is a personal part of the such a personal promise the such an actions of the strength and in the strength and the more in the such a such a such a soul of the action of the concealed and as a more more and the such a strength of the same all the such a personal part of the such a concealed and in the same experience of the such a personal part of the such a 
-------------- temperature: 0.5
e same experience of the such a personal part of the such a such a personal fact that that it is the most families the most personal, he is a habitually has even in the fact is with the belief and man of the profuls, as for one which is long only the conceptions of the extend to this such the highest be be

13it assumernest:
on the sunearamen conposiing ceking--when one meass
irmuse her medes onee--longd raterner, goal, world cour sheble wou? brary itself aremoss cake: how without the further metaphysical,
yet realings, good, as he fundamentally eechen lads was by uetarnis, in souls" "promiss to even chenchokr knoe of regarded people festal
called which a man froe untrooks cause is all nothingway a ber
epoch 10
Epoch 1/1
200278/200278 [==============================] - 67s 332us/step - loss: 1.3719
--- Generating with seed: "ly
their heart's desire abstracted and refined, is defended "
-------------- temperature: 0.2
ly
their heart's desire abstracted and refined, is defended the states and states and protection of the best of the states and strength and profound and more states the states the religious stranges of the states and stranger of the states and more according the states and still of the states and according the strength of the state of the good and states and states and conseq

...

cherald y in test nhe t s ioin taitnne n  di tei  ate  thrheitar ine ca cledel  xuo irte thdeor 
noon thoofnidihidt alee s llou lon ir dd ic is anirsio i s d i he tae dlyteinnsldad d tithe ntæeenlu  deof o ninohs  eutesedtht   d turaneed indeatinltnsesa ad cqutett o adsatectut  n uo cen vdindhacn  ddo oi onflan  einde  d yateeil dnt oirthanndeerreindnd nn he ke th  s rdedlcta  lle .st h hedoundh unin  ootanrnhrd  aoict(ut cre a oalfiitheranoraca cahs  toes
-------------- temperature: 1.2
nin  ootanrnhrd  aoict(ut cre a oalfiitheranoraca cahs  toese"tore  a  th dizatttitonutehtonirn cin hs  deen irue dofisthnic ternhe atad isje ranatet anas uun  thantend nhen nd cselnof dcanthi sri aershsctine eitsrh  aetd  ? int ath sinnaetheesidea ia eintoustland theuie alndrie t hr ithndinece sdtrruce a sn thu sslhnrien e lid "noen da iert a r rhi oos theoo nrih ultlle  enuoiae isnil tes tt ieed annsid ndd enoordecc hd iltde turrid tsthreeootsaat l lin e
epoch 44
Epoch 1/1
200278/200278 [============

tosess  is s ose eies in sh athe dis  at ais oes e a n  hs s rhrinot th i asioes. is  s heeses edmces ls araad henie id se ceonse thessiece sitecesh ed sf irts so inson thisoseson os s sshs eis he ans enoe( soton sho sree  isise is aosse asis ishered ti seitte st sesshi  esouieesoi a si  se thsee anioeded ansas otfeoust e otiss sedinaces cia ot ti hed toefnd inssotees etheutil as hese rens thsiode nca sas en co resendseh s e  t is s teistas hs in seilnse s
-------------- temperature: 1.0
 nca sas en co resendseh s e  t is s teistas hs in seilnse sd ndndntoionsonesurausceti ho s alhe nsusohdessortedaestenc tharuacoatnhdalsuos tossessicorea thesahuaisd codadtedi tsaainorh odes ttoido y'sas o odn r osse h hee ceneen tein nt  ncd th incoellu dsestxesr iaro auoenes osacrn onis seesitois as c scoolc tiietn esiodd   cson u iielalrdsthslln arhedburetnv illuseeeonoinose atoselbe yheteastethnd siiouthai or esn ern thi thec  itotchcs t cilaene so art
-------------- temperature: 1.2
hnd siiouthai 

to my ears t n ao  n t ea th no t  e t " an  ! oek o e t tn the oten e th eethe r t  tnhe t x t tat tthe t   t ate tne te t itho nente   f t tr   th  an he an t t thet e   e tt ee as   ti   tt tta t e  tveiet n en f theees  tthe ) f tot  teent an   tr t tan  an t an  t   he t tot th oen "th ret t
t an t x as t t t othe  othe an jteoe  ta thet etir'enbe t" g  the tvn tine  t e  oe on ans the nno e  at tf tt 
-------------- temperature: 0.5
enbe t" g  the tvn tine  t e  oe on ans the nno e  at tf tt t"herein na e hn s tneded t t  thect  n o  nhillr tan ih o
stctuob te at a es t nhe  t or n s thino" onsfexithe xton othodtiee t o  th te"sothneaninend t sane th ooete"elen an ant shv aqlooijt  iconta t tha ni fnothetoeanlrf a titetu"enerw ooan todai o an inenn d ais uhe st toezicitn  txeo o  antth o  tae hie  otetat t:hehit x  esw tfou tetoe  initntoeuanhethee thulcdesen  t t e ijnhon nt  eosfo t
-------------- temperature: 1.0
etoe  initntoeuanhethee thulcdesen  t t e ijnhon nt  eosfo t itl

sre lirylcoehitsaeansoaest y iocs ooe oc nn e  dtnioeaeh o ens lce snn siotd shs oqtyeelse i t
rd 
sielyhiif rhcofat itll ceahcs anhf  us
ot sednds a hess
ahts chtci   i
eth iataslesb ho
epoch 55
Epoch 1/1
200278/200278 [==============================] - 66s 328us/step - loss: 4.5940
--- Generating with seed: "come interesting! but let us not be afraid! things still rem"
-------------- temperature: 0.2
come interesting! but let us not be afraid! things still rem et an   ithes hn stheeeo  an tin s ter aie t   nt n  ene  e lnennt r hnen f eath thhe t   he rner t as esgh n n intbe ator tir thn ins 
tie nr ttoln  rertqen ntr et r tte  re nt t athrr in antt de inn  t t e e  j   n hnri    a tt inof the  tha  iztn t  tsb ts  en an tn thi an  innant  ne etoanelre tonent"o t nt  an  inonan  t int efjnnt  e  on t ro aan fat to  thrre tt ter  tin   ne  nen t in t t
-------------- temperature: 0.5
 e  on t ro aan fat to  thrre tt ter  tin   ne  nen t in t th at t 
tt  anerisnitthsi ti e tay tedt  

etaeuf s nws lentnut eru rro thlseu n xlnlr thfeihycfneharr ssiytre  o nuth oo i dto(hartroioyt ut y cy ldoth e oashtlfeius terdlid ulo eeo dt td
-------------- temperature: 1.2
hartroioyt ut y cy ldoth e oashtlfeius terdlid ulo eeo dt tdtreilift   nl fates  tt
ra ro csaida urd d
o tlachaa
  athaatetehr h icdothftsryancailnlyoino oaeartthente niosoascdhelondoee
stsn odetheliss e atie foinofyhe nahouuirnaeefeur do y rl t helioanhe
ee
hluu  nrstfbedlrqnyira
du
e
 euonof

hl casrrauhdusiihfntcees  olotnlddurdonatiseeliyis  rtor chete netinyaefir r 
ideaaoh  x ds ed
cihaeo sieeh a
niilinionxahc"thait
aeirel hrta dillue ase ctafrseasa 
epoch 59
Epoch 1/1
200278/200278 [==============================] - 65s 325us/step - loss: 4.3764
--- Generating with seed: " it is preserved because of its supposed vital utility. it i"
-------------- temperature: 0.2
 it is preserved because of its supposed vital utility. it is  s  dite  ano r   oreaoe  tee t tthe d   n ee toeat  the onet e of neseee s oe" 